In [1]:
# !pip install -U -q PyDrive

# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# # Authenticate and create the PyDrive client.
# # This only needs to be done once per notebook.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# file_id = '0BzQ6rtO2VN95a0c3TlZCWkl3aU0' # URL id. 
# downloaded = drive.CreateFile({'id': file_id})
# downloaded.GetContentFile('finished_files.zip')
# !unzip finished_files.zip

# !pip install pyrouge
# !mkdir log
# !pip install tensorflow==1.15.0
# !pip install tensorflow-gpu==1.15.0

In [2]:
import torch

import config
from numpy import random
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence
from torch.nn.utils.rnn import pack_padded_sequence
import tensorflow as tf
import math

use_cuda = config.use_gpu and torch.cuda.is_available()

random.seed(123)
torch.manual_seed(123)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(123)

In [3]:
class BasicModule(nn.Module):
    def __init__(self, init='uniform'):
        super(BasicModule, self).__init__()
        self.init = init

    def init_params(self):
        for param in self.parameters():
            if param.requires_grad and len(param.shape) > 0:
                stddev = 1 / math.sqrt(param.shape[0])
                if self.init == 'uniform':
                    torch.nn.init.uniform_(param, a=-0.05, b=0.05)
                elif self.init == 'normal':
                    torch.nn.init.normal_(param, std=stddev)
                elif self.init == 'truncated_normal':
                    self.truncated_normal_(param, mean=0,std=stddev)

    def truncated_normal_(self, tensor, mean=0, std=1.):
        size = tensor.shape
        tmp = tensor.new_empty(size + (4,)).normal_()
        valid = (tmp < 2) & (tmp > -2)
        ind = valid.max(-1, keepdim=True)[1]
        tensor.data.copy_(tmp.gather(-1, ind).squeeze(-1))
        tensor.data.mul_(std).add_(mean)
        return tensor

In [4]:
class Model(object):
    def __init__(self, model_path=None, is_eval=False, is_tran = False):
        encoder = Encoder()
        decoder = Decoder()
        reduce_state = ReduceState()
        if is_tran:
            encoder = TranEncoder(config.vocab_size, config.max_enc_steps, config.emb_dim,
                 config.n_layers, config.n_head, config.d_k, config.d_v, config.d_model, config.d_inner)

        # shared the embedding between encoder and decoder
        decoder.tgt_word_emb.weight = encoder.src_word_emb.weight

        if is_eval:
            encoder = encoder.eval()
            decoder = decoder.eval()
            reduce_state = reduce_state.eval()

        if use_cuda:
            encoder = encoder.cuda()
            decoder = decoder.cuda()
            reduce_state = reduce_state.cuda()

        self.encoder = encoder
        self.decoder = decoder
        self.reduce_state = reduce_state

        if model_path is not None:
            state = torch.load(model_path, map_location=lambda storage, location: storage)
            self.encoder.load_state_dict(state['encoder_state_dict'])
            self.decoder.load_state_dict(state['decoder_state_dict'], strict=False)
            self.reduce_state.load_state_dict(state['reduce_state_dict'])


In [5]:
class Encoder(BasicModule):
    def __init__(self):
        super(Encoder, self).__init__()
        self.src_word_emb = nn.Embedding(config.vocab_size, config.emb_dim)
        self.lstm = nn.LSTM(config.emb_dim, config.hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(config.hidden_dim * 2, config.hidden_dim * 2, bias=False)

        self.init_params()

    # seq_lens should be in descending order
    def forward(self, input, seq_lens):
        embedded = self.src_word_emb(input)

        packed = pack_padded_sequence(embedded, seq_lens, batch_first=True)
        output, hidden = self.lstm(packed)

        encoder_outputs, _ = pad_packed_sequence(output, batch_first=True)  # h dim = B x l x n
        encoder_outputs = encoder_outputs.contiguous()

        encoder_feature = encoder_outputs.view(-1, 2 * config.hidden_dim)   # B*l x 2*hidden_dim
        encoder_feature = self.fc(encoder_feature)

        return encoder_outputs, encoder_feature, hidden

In [6]:
class Attention(BasicModule):
    def __init__(self):
        super(Attention, self).__init__()

        self.fc = nn.Linear(config.hidden_dim * 2, 1, bias=False)
        self.dec_fc = nn.Linear(config.hidden_dim * 2, config.hidden_dim * 2)
        if config.is_coverage:
            self.con_fc = nn.Linear(1, config.hidden_dim * 2, bias=False)

        self.init_params()

    def forward(self, s_t, enc_out, enc_fea, enc_padding_mask, coverage):
        b, l, n = list(enc_out.size())

        dec_fea = self.dec_fc(s_t)  # B x 2*hidden_dim
        dec_fea_expanded = dec_fea.unsqueeze(1).expand(b, l, n).contiguous()  # B x l x 2*hidden_dim
        dec_fea_expanded = dec_fea_expanded.view(-1, n)     # B*l x 2*hidden_dim

        att_features = enc_fea + dec_fea_expanded           # B*l x 2*hidden_dim
        if config.is_coverage:
            coverage_inp = coverage.view(-1, 1)             # B*l x 1
            coverage_fea = self.con_fc(coverage_inp)        # B*l x 2*hidden_dim
            att_features = att_features + coverage_fea

        e = torch.tanh(att_features)                        # B*l x 2*hidden_dim
        scores = self.fc(e)                                 # B*l x 1
        scores = scores.view(-1, l)                         # B x l

        attn_dist_ = F.softmax(scores, dim=1) * enc_padding_mask  # B x l
        normalization_factor = attn_dist_.sum(1, keepdim=True)
        attn_dist = attn_dist_ / normalization_factor

        attn_dist = attn_dist.unsqueeze(1)                        # B x 1 x l
        c_t = torch.bmm(attn_dist, enc_out)                       # B x 1 x n
        c_t = c_t.view(-1, config.hidden_dim * 2)                 # B x 2*hidden_dim

        attn_dist = attn_dist.view(-1, l)                         # B x l

        if config.is_coverage:
            coverage = coverage.view(-1, l)
            coverage = coverage + attn_dist

        return c_t, attn_dist, coverage

In [7]:
class ReduceState(BasicModule):
    def __init__(self):
        super(ReduceState, self).__init__()

        self.reduce_h = nn.Linear(config.hidden_dim * 2, config.hidden_dim)
        self.reduce_c = nn.Linear(config.hidden_dim * 2, config.hidden_dim)
        self.init_params()


    def forward(self, hidden):
        h, c = hidden  # h, c dim = 2 x b x hidden_dim
        h_in = h.transpose(0, 1).contiguous().view(-1, config.hidden_dim * 2)
        hidden_reduced_h = F.relu(self.reduce_h(h_in))
        c_in = c.transpose(0, 1).contiguous().view(-1, config.hidden_dim * 2)
        hidden_reduced_c = F.relu(self.reduce_c(c_in))

        return (hidden_reduced_h.unsqueeze(0), hidden_reduced_c.unsqueeze(0))  # h, c dim = 1 x b x hidden_dim

In [8]:
class Decoder(BasicModule):
    def __init__(self):
        super(Decoder, self).__init__()
        self.attention_network = Attention()
        # decoder
        self.tgt_word_emb = nn.Embedding(config.vocab_size, config.emb_dim)
        self.con_fc = nn.Linear(config.hidden_dim * 2 + config.emb_dim, config.emb_dim)
        self.lstm = nn.LSTM(config.emb_dim, config.hidden_dim, batch_first=True, bidirectional=False)

        if config.pointer_gen:
            self.p_gen_fc = nn.Linear(config.hidden_dim * 4 + config.emb_dim, 1)

        # p_vocab
        self.fc1 = nn.Linear(config.hidden_dim * 3, config.hidden_dim)
        self.fc2 = nn.Linear(config.hidden_dim, config.vocab_size)

        self.init_params()

    def forward(self, y_t, s_t, enc_out, enc_fea, enc_padding_mask,
                c_t, extra_zeros, enc_batch_extend_vocab, coverage, step):

        if not self.training and step == 0:
            dec_h, dec_c = s_t
            s_t_hat = torch.cat((dec_h.view(-1, config.hidden_dim),
                                 dec_c.view(-1, config.hidden_dim)), 1)  # B x 2*hidden_dim
            c_t, _, coverage_next = self.attention_network(s_t_hat, enc_out, enc_fea,
                                                           enc_padding_mask, coverage)
            coverage = coverage_next

        y_t_embd = self.tgt_word_emb(y_t)
        x = self.con_fc(torch.cat((c_t, y_t_embd), 1))
        lstm_out, s_t = self.lstm(x.unsqueeze(1), s_t)

        dec_h, dec_c = s_t
        s_t_hat = torch.cat((dec_h.view(-1, config.hidden_dim),
                             dec_c.view(-1, config.hidden_dim)), 1)     # B x 2*hidden_dim
        c_t, attn_dist, coverage_next = self.attention_network(s_t_hat, enc_out, enc_fea,
                                                               enc_padding_mask, coverage)

        if self.training or step > 0:
            coverage = coverage_next

        p_gen = None
        if config.pointer_gen:
            p_gen_inp = torch.cat((c_t, s_t_hat, x), 1)  # B x (2*2*hidden_dim + emb_dim)
            p_gen = self.p_gen_fc(p_gen_inp)
            p_gen = torch.sigmoid(p_gen)

        output = torch.cat((lstm_out.view(-1, config.hidden_dim), c_t), 1)  # B x hidden_dim * 3
        output = self.fc1(output)  # B x hidden_dim
        # output = F.relu(output)

        output = self.fc2(output)  # B x vocab_size
        vocab_dist = F.softmax(output, dim=1)

        if config.pointer_gen:
            vocab_dist_ = p_gen * vocab_dist
            attn_dist_ = (1 - p_gen) * attn_dist

            if extra_zeros is not None:
                vocab_dist_ = torch.cat([vocab_dist_, extra_zeros], 1)

            final_dist = vocab_dist_.scatter_add(1, enc_batch_extend_vocab, attn_dist_)
        else:
            final_dist = vocab_dist

        return final_dist, s_t, c_t, attn_dist, p_gen, coverage

In [13]:
from dataset import Vocab
from dataset import Batcher
from utils import get_input_from_batch
from utils import get_output_from_batch
from utils import calc_running_avg_loss
import torch.optim as optim
from torch.nn.utils import clip_grad_norm_
import time
import os

In [10]:
use_cuda = config.use_gpu and torch.cuda.is_available()

In [15]:
class Train(object):
    def __init__(self):
        print(config.vocab_path)
        self.vocab = Vocab(config.vocab_path, config.vocab_size)
        self.batcher = Batcher(self.vocab, config.train_data_path,
                               config.batch_size, single_pass=False, mode='train')
        time.sleep(10)

        train_dir = os.path.join(config.log_root, 'train_%d' % (int(time.time())))
        if not os.path.exists(train_dir):
            os.mkdir(train_dir)

        self.model_dir = os.path.join(train_dir, 'models')
        if not os.path.exists(self.model_dir):
            os.mkdir(self.model_dir)

        self.summary_writer = tf.summary.FileWriter(train_dir)

    def save_model(self, running_avg_loss, iter):
        state = {
            'iter': iter,
            'encoder_state_dict': self.model.encoder.state_dict(),
            'decoder_state_dict': self.model.decoder.state_dict(),
            'reduce_state_dict': self.model.reduce_state.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'current_loss': running_avg_loss
        }
        model_save_path = os.path.join(self.model_dir, 'model_%d_%d' % (iter, int(time.time())))
        torch.save(state, model_save_path)

    def setup_train(self, model_path=None):
        self.model = Model(model_path)
        initial_lr = config.lr_coverage if config.is_coverage else config.lr

        params = list(self.model.encoder.parameters()) + list(self.model.decoder.parameters()) + \
                 list(self.model.reduce_state.parameters())
        total_params = sum([param[0].nelement() for param in params])
        print('The Number of params of model: %.3f million' % (total_params))  # million
        self.optimizer = optim.Adagrad(params, lr=initial_lr, initial_accumulator_value=config.adagrad_init_acc)

        start_iter, start_loss = 0, 0

        if model_path is not None:
            state = torch.load(model_path, map_location=lambda storage, location: storage)
            start_iter = state['iter']
            start_loss = state['current_loss']

            if not config.is_coverage:
                self.optimizer.load_state_dict(state['optimizer'])
                if use_cuda:
                    for state in self.optimizer.state.values():
                        for k, v in state.items():
                            if torch.is_tensor(v):
                                state[k] = v.cuda()

        return start_iter, start_loss

    def train_one_batch(self, batch):
        enc_batch, enc_lens, enc_pos, enc_padding_mask, enc_batch_extend_vocab, \
        extra_zeros, c_t, coverage = get_input_from_batch(batch, use_cuda)
        dec_batch, dec_lens, dec_pos, dec_padding_mask, max_dec_len, tgt_batch = \
            get_output_from_batch(batch, use_cuda)

        self.optimizer.zero_grad()

        if not config.tran:
            enc_out, enc_fea, enc_h = self.model.encoder(enc_batch, enc_lens)
        else:
            enc_out, enc_fea, enc_h = self.model.encoder(enc_batch, enc_pos)

        s_t = self.model.reduce_state(enc_h)

        step_losses, cove_losses = [], []
        for di in range(min(max_dec_len, config.max_dec_steps)):
            y_t = dec_batch[:, di]  # Teacher forcing
            final_dist, s_t, c_t, attn_dist, p_gen, next_coverage = \
                self.model.decoder(y_t, s_t, enc_out, enc_fea, enc_padding_mask, c_t,
                                   extra_zeros, enc_batch_extend_vocab, coverage, di)
            tgt = tgt_batch[:, di]
            step_mask = dec_padding_mask[:, di]
            gold_probs = torch.gather(final_dist, 1, tgt.unsqueeze(1)).squeeze()
            step_loss = -torch.log(gold_probs + config.eps)
            if config.is_coverage:
                step_coverage_loss = torch.sum(torch.min(attn_dist, coverage), 1)
                step_loss = step_loss + config.cov_loss_wt * step_coverage_loss
                cove_losses.append(step_coverage_loss * step_mask)
                coverage = next_coverage

            step_loss = step_loss * step_mask
            step_losses.append(step_loss)

        sum_losses = torch.sum(torch.stack(step_losses, 1), 1)
        batch_avg_loss = sum_losses / dec_lens
        loss = torch.mean(batch_avg_loss)

        loss.backward()

        clip_grad_norm_(self.model.encoder.parameters(), config.max_grad_norm)
        clip_grad_norm_(self.model.decoder.parameters(), config.max_grad_norm)
        clip_grad_norm_(self.model.reduce_state.parameters(), config.max_grad_norm)

        self.optimizer.step()

        if config.is_coverage:
            cove_losses = torch.sum(torch.stack(cove_losses, 1), 1)
            batch_cove_loss = cove_losses / dec_lens
            batch_cove_loss = torch.mean(batch_cove_loss)
            return loss.item(), batch_cove_loss.item()

        return loss.item(), 0.

    def run(self, n_iters, model_path=None):
        iter, running_avg_loss = self.setup_train(model_path)
        start = time.time()
        interval = 100

        while iter < n_iters:
            batch = self.batcher.next_batch()
            loss, cove_loss = self.train_one_batch(batch)

            running_avg_loss = calc_running_avg_loss(loss, running_avg_loss, self.summary_writer, iter)
            iter += 1

            if iter % interval == 0:
                self.summary_writer.flush()
                print(
                    'step: %d, second: %.2f , loss: %f, cover_loss: %f' % (iter, time.time() - start, loss, cove_loss))
                start = time.time()
            if iter % 5000 == 0:
                self.save_model(running_avg_loss, iter)



In [16]:
train_processor = Train()
train_processor.run(config.max_iterations)

./finished_files/vocab
HERERERERERE
Finished constructing vocabulary of 50 total words. Last word added: their
INFO:tensorflow:Bucket queue size: 15, Input queue size: 0
The Number of params of model: 314.000 million


In [ ]:
!ls

In [ ]:
import config
print(config.beam_size)

In [ ]:
!cat config.py